# Pre-processing of GDMP 100m time-series  raster dataset for the uploaded to JEDI

The following notbooks set up to help raster-data-processing for the uploaded to JEDI
https://jedi.discomap.eea.europa.eu/Search

Requirements for raster upload to JEDI: 
- pixel sizes in (10m, 100, 1000m or 10 000m) 
- projection: LAEA -EPSG3035 
- datatype: INT8,UINT8,INT16 ant UINT16.
- pixel nbit proberty:  NO nbits--> remove NBITS 
- Compression:  -co COMPRESS=JPEG -co TILED=YES
- format: *.tif


Every raster in the zip will be added as an attribute to the dimension.
You can exclude values inside the JEDI website under the field options. Provide a value seperated list of all values you wish to exclude.
The exclude will only work for INT8,UINT8,INT16 ant UINT16.

So, there is a workaround in place:
1. Create a zip with the same tifs but these must be clipped to a small area.
This way the uploaded zip is small but enough to populate the schema in the db.
2. When the dim is created (but not completed) send us the path to the real zip file and we will take care of it



## (Tool 1 -A) small subset to prepare the jedi upload *using a small clip


L:\f02_data\GDMP\input\GDMP_100m

In [2]:
# tool1 layerstack to single band
import os, sys 
import arcpy
from arcpy.sa import *
from pathlib import Path
import os
import sys
from datetime import datetime
import multiprocessing
import glob
import zipfile
import shutil  ## for zipping

#L:\f02_data\GDMP\input\GDMP_100m
# set data folder
sys.path.append(Path(os.getcwd()).parent.joinpath('src').as_posix())
sys.path.append(Path(os.getcwd()).joinpath('src').as_posix())
from constants import dir_signature

###################################################################################################################################### SET
base_path = os.path.join(dir_signature, 'GDMP')
base_path_input = os.path.join(base_path, 'input','GDMP_100m') # , 'SOC_g_m2_90-21')   ###inputs
#outzip = OutRas+'zip'


zip_folder = r'L:\f02_data\GDMP\output\GDMP_100m_dask\jedi\gdmp_time_series'
raster_folder_with_final_raster_datasets = r'L:\f02_data\GDMP\input\GDMP_1km\v2'
aoi_folder = r'L:\f02_data\GDMP\input\GDMP_100m\aoi'
aoi_folder_out =r'L:\f02_data\GDMP\output\GDMP_100m_dask\jedi\gdmp_time_series\aoi'
###############################################output info:Start 
pixel_type_info ='16_BIT_SIGNED'
format_info = 'COG'
NoData_info = -999
###############################################output info:End

# To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True
first_year = 1990  # to put the year into the raster name 
###################################################################################################################################### SET
print ("START.................")


arcpy.env.workspace = base_path_input
rasterlist  = arcpy.ListRasters()
# ... loop over all tif files in folder:
for raster in rasterlist:
        print (raster)
        print ("..will be updated and exoprted....")
        ## PART1 export raster with correct nuldata, pixel type and pixel size:####################
        InRaster = raster
        output_raster_name= raster_folder_with_final_raster_datasets+"\\"+raster.rsplit('.', maxsplit=1)[0]+'_v2.tif'
        OutRaster = output_raster_name
        arcpy.CopyRaster_management(InRaster,OutRaster,nodata_value = NoData_info,pixel_type=pixel_type_info,format=format_info)
        ### zipping
        outzip=zip_folder+"\\"+raster.rsplit('.', maxsplit=1)[0]+'.zip'
        with zipfile.ZipFile(outzip, 'w') as f:#
            f.write(output_raster_name, os.path.basename(output_raster_name))


        print ("..will be clipped and exoprted....")
        #print (output_raster_name)
        ## PART2 export clip raster##################################################################
        #aoi (luxembourg)
        extent = "3015500 2933800 4014700 4071300"
        InRaster = OutRaster
        output_raster_name= aoi_folder+"\\"+raster.rsplit('.', maxsplit=1)[0]+'_clip.tif'
        arcpy.Clip_management(InRaster, extent, output_raster_name)

        ### zipping aoi
        outzip=zip_folder+"\\"+raster.rsplit('.', maxsplit=1)[0]+'.zip'
        with zipfile.ZipFile(outzip, 'w') as f:#
            f.write(output_raster_name, os.path.basename(output_raster_name))
        
       #     raster = gdal.Open(raster.name)
       #     bands = raster.RasterCount




print ("done")

ModuleNotFoundError: No module named 'geopandas'

## (Tool 1 -B) FULL production of JEDI upload: